In [1]:
!pip install -r requirements.txt


In [2]:
pip install requests beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from bs4 import BeautifulSoup as soup
import csv
from urllib.parse import quote_plus
import json


# 1. Scraping the list page

In [4]:
import requests
from bs4 import BeautifulSoup

event_urls = []

for page in range(1, 41):
    url = f"https://visitseattle.org/events/?page={page}"
    res = requests.get(url)
    if res.status_code == 200:
        # Create a BeautifulSoup object for each page
        soup = BeautifulSoup(res.content, 'html.parser')
        selector = "div > div.container-event > h4 > a"
        events = soup.select(selector)
        for event in events:
            event_url = event['href']
            event_urls.append(event_url)
            print(event_url)


https://visitseattle.org/events/2024-kpmg-womens-pga-championship/
https://visitseattle.org/events/2024-university-of-washington-mfa-mdes-thesis-exhibition/
https://visitseattle.org/events/6th-annual-sami-film-festival/
https://visitseattle.org/events/75-years-of-porsche/
https://visitseattle.org/events/a-case-for-the-existence-of-god/
https://visitseattle.org/events/a-tale-of-peter-rabbit/
https://visitseattle.org/events/a-political-rocks/
https://visitseattle.org/events/adira-sharkey/
https://visitseattle.org/events/ajr/
https://visitseattle.org/events/al-youngs-1970-dodge-challenger-drag-racer/
https://visitseattle.org/events/alice-in-shadowland/
https://visitseattle.org/events/all-will-suffer/
https://visitseattle.org/events/amazing-life/
https://visitseattle.org/events/american-art-the-stories-we-carry/
https://visitseattle.org/events/american-flats-album-release/
https://visitseattle.org/events/an-evening-with-jinjoo-cho/
https://visitseattle.org/events/an-evening-with-tina-campt

# 2. Scraping the detail page

In [5]:
import requests
from bs4 import BeautifulSoup
import json

event_urls = []

for page in range(1, 41):
    url = f"http://visitseattle.org/events/?page={page}"
    res = requests.get(url)

    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        selector = "div > div.container-event > h4 > a"
        events = soup.select(selector)
        
        for event in events:
            event_url = event['href']
            print(event.text)  # Print the event text
            event_urls.append(event_url)  # Collect the event URL

# Dump all collected event URLs into a JSON file
with open('events.json', 'w') as file:
    json.dump(event_urls, file)

print("Event URLs saved to events.json")



2024 KPMG Women’s PGA Championship
2024 University of Washington MFA + MDes Thesis Exhibition
6th Annual Sami Film Festival
75 Years of Porsche
A Case for the Existence of God
A Tale of Peter Rabbit
A/political Rocks
Adira Sharkey
AJR
Al Young’s 1970 Dodge Challenger Drag Racer
Alice in Shadowland
All Will Suffer
Amazing Life
American Art: The Stories We Carry
American Flats Album Release
An Evening with Jinjoo Cho
An Evening with Tina Campt: Black Feminist Visual Culture
Anand Gopal
Anida Yoeu Ali: Hybrid Skin, Mythical Presence
Annie Eastwood with Kimball and the Fugitives
Antarctic Symphony
Antonio M. Gomez: LINEAJES
Apollo
Aqua Fools’ Day
2024 KPMG Women’s PGA Championship
2024 University of Washington MFA + MDes Thesis Exhibition
6th Annual Sami Film Festival
75 Years of Porsche
A Case for the Existence of God
A Tale of Peter Rabbit
A/political Rocks
Adira Sharkey
AJR
Al Young’s 1970 Dodge Challenger Drag Racer
Alice in Shadowland
All Will Suffer
Amazing Life
American Art: The Sto

### Data written to csv file

In [6]:
# Function to scrape event details
def scrape_event_details(url):
    res = requests.get(url)
   ##if res.status_code == 200:
    soup = BeautifulSoup(res.text, 'html.parser')
    name = soup.select_one('div.medium-6.columns.event-top > h1').get_text(strip=True)
    date = soup.select_one('div.medium-6.columns.event-top > h4 > span:nth-child(1)').get_text(strip=True)
    location = soup.select_one('div.medium-6.columns.event-top > h4 > span:nth-child(2)').get_text(strip=True)
    event_type = soup.select_one('div.medium-6.columns.event-top > a:nth-child(3)').get_text(strip=True)
    region = soup.select_one('div.medium-6.columns.event-top > a:nth-child(4)').get_text(strip=True)

    return [name, date, location, event_type, region]
    ##else:
    ##    return ['N/A', 'N/A', 'N/A', 'N/A', 'N/A']  # In case the page request fails

# Part 1: Collecting URLs and Writing to 'events.json'

event_urls = []

# Collecting URLs
for page in range(1, 41):
    url = f"http://visitseattle.org/events/?page={page}"
    res = requests.get(url)

    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        event_links = soup.select('div > div.container-event > h4 > a')

        for event_link in event_links:
            event_url = event_link['href']
            event_urls.append(event_url)

# Writing URLs to 'events.json'
with open('events.json', 'w') as file:
    json.dump(event_urls, file)

print("Event URLs saved to events.json")

# Part 2: Reading URLs from 'events.json', Scraping Details, and Writing to 'events.csv'

# Reading URLs from 'events.json'
with open('events.json', 'r') as file:
    event_urls = json.load(file)

# Scraping event details
event_data = []
for event_url in event_urls:
    details = scrape_event_details(event_url)
    event_data.append(details)

# Writing data to 'events.csv'
with open('events.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Date', 'Location', 'Type', 'Region'])
    writer.writerows(event_data)

print("Data written to events.csv")


Event URLs saved to events.json
Data written to events.csv


### Look up the location

In [13]:
import csv
import requests

# Function to get latitude and longitude for a given location using Nominatim API
def get_lat_lon(location):
    base_url = "https://nominatim.openstreetmap.org/search.php"
    query_params = {
        "q": location,
        "format": "jsonv2"
    }
    try:
        response = requests.get(base_url, params=query_params)
        response.raise_for_status()
        data = response.json()
        if data:
            return data[0].get('lat'), data[0].get('lon')
        else:
            return None, None
    except Exception as e:
        print(f"Error during geocoding: {e}")
        return None, None

# Path to the original CSV file
input_file_path = 'events.csv'
# Path to the new CSV file with latitudes and longitudes
output_file_path = 'events_with_lat_lon.csv'

# Read the CSV, fetch geolocation data, and write to a new CSV
with open(input_file_path, mode='r', encoding='utf-8') as infile, \
     open(output_file_path, mode='w', newline='', encoding='utf-8') as outfile:

    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames + ['Latitude', 'Longitude']  # Add new columns
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        lat, lon = get_lat_lon(row['Location'])
        row['Latitude'] = lat
        row['Longitude'] = lon
        writer.writerow(row)

print("Updated data written to events_with_lat_lon.csv")


Error during geocoding: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search.php?q=The+Museum+of+Flight&format=jsonv2 (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))
Error during geocoding: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search.php?q=The+Royal+Room&format=jsonv2 (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))


### Look up the weather for the location

In [ ]:
import csv
import requests
from datetime import datetime

# Function to get weather forecast for given latitude and longitude
def get_weather_forecast(latitude, longitude, event_date):
    url = f"https://api.weather.gov/points/{latitude},{longitude}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        point_data = response.json()
        forecast_url = point_data['properties']['forecast']
        forecast_response = requests.get(forecast_url)
        forecast_response.raise_for_status()
        forecast_data = forecast_response.json()
        
        for period in forecast_data['properties']['periods']:
            period_start = datetime.fromisoformat(period['startTime'].replace('Z', '+00:00'))
            if period_start.date() == event_date and period['isDaytime']:
                return period['detailedForecast']
        return 'Daytime forecast not available'
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return 'Weather data not available'

# Function to extract the start date from a date range string
def extract_start_date(date_str):
    # Skip non-standard date formats like 'Now' or 'Ongoing'
    if date_str in ['Now', 'Ongoing']:
        return None
    try:
        return datetime.strptime(date_str.split(' through ')[0], '%m/%d/%Y').date()
    except ValueError:
        # If the date format is not recognized, skip this row
        return None



### Store the data as CSV

In [9]:
# Path to the original CSV file and the new CSV file with weather data
input_file_path = 'events_with_lat_lon.csv'
output_file_path = 'events_with_wea.csv'

# Read the CSV, fetch weather data, and write to a new CSV
with open(input_file_path, mode='r', encoding='utf-8') as infile, \
     open(output_file_path, mode='w', newline='', encoding='utf-8') as outfile:

    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames + ['Weather']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        event_date = extract_start_date(row['Date'])
        if event_date is None:
            continue  # Skip row if the date is invalid or non-standard

        latitude = row['Latitude']
        longitude = row['Longitude']
        # Ensure latitude and longitude are valid before fetching weather data
        if not latitude or not longitude:
            row['Weather'] = 'Weather data not available'
            writer.writerow(row)
            continue

        weather_forecast = get_weather_forecast(latitude, longitude, event_date)
        row['Weather'] = weather_forecast
        writer.writerow(row)

print("Updated data written to events_with_wea.csv")

Error fetching weather data: HTTPSConnectionPool(host='api.weather.gov', port=443): Max retries exceeded with url: /points/32.2275723,-110.9685178 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))
Updated data written to events_with_wea.csv


# Reference 

In [10]:
# Location API

url = "https://api.weather.gov/points/39.7456,-97.0892"
res = requests.get(url)
res
res.text
point_dict = res.json()
point_dict
point_dict.keys()
forcast_url = point_dict['properties']['forecast']
forcast_url
res = requests.get(forcast_url)
res.json()

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-97.1089731, 39.7668263],
    [-97.1085269, 39.7447788],
    [-97.0798467, 39.7451195],
    [-97.08028680000001, 39.767167],
    [-97.1089731, 39.7668263]]]},
 'properties': {'updated': '2024-01-28T04:40:32+00:00',
  'units': 'us',
  'forecastGenerator': 'BaselineForecastGenerator',
  'generatedAt': '2024-01-28T07:15:51+00:00',
  'updateTime': '2024-01-28T04:40:32+00:00',
  'validTimes': '2024-01-27T22:00:00+00:00/P7DT6H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 441.96},
  'periods': [{'number': 1,
    'name': 'Overnight',
    'startTime': '2024-01-28T01:00:00-06:00',
    'endTime': '2024-01-28T06:00:00-06:00',
    'isD

In [11]:
#OpenStreeetMap Geocoding API

# location name to lat lon

# Option 1: just string manipulation
base_url = "https://nominatim.openstreetmap.org/search.php"

# query_params_str = "?q=Wallingford%2C+Seattle&format=jsonv2"
# ful_url = base_url + query_params_str

# Option 2: use dictionary to represent query params
#           use requests.get(url, params=query_params) to attach the query param dict

query_params = {
    "q": "Wallingford, Seattle",
    "format": "jsonv2"
}

res = requests.get(base_url, params=query_params)
res.json()

[{'place_id': 312844859,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 150973716,
  'lat': '47.6594631',
  'lon': '-122.3343417',
  'category': 'place',
  'type': 'suburb',
  'place_rank': 19,
  'importance': 0.3131791431958073,
  'addresstype': 'suburb',
  'name': 'Wallingford',
  'display_name': 'Wallingford, Seattle, King County, Washington, 98015, United States',
  'boundingbox': ['47.6394631', '47.6794631', '-122.3543417', '-122.3143417']}]